# Deploy your model on a SageMaker Endpoint and predict

## Setup environment

In [ ]:
import numpy as np
import pandas as pd
import sagemaker
import matplotlib.pyplot as plt
from sagemaker.model import Model
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from deepAR_predictor import DeepARPredictor


np.random.seed(1)
sagemaker_session = sagemaker.Session()
region = sagemaker.Session().boto_region_name
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
model_artefact = '<your-model-s3-path>'

## Create endpoint and predictor

In [ ]:
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, 'forecasting-deepar', 'latest')

model = Model(model_data=model_artefact,
              image=image_name,
              name=name_from_base('forecasting-model'),
              role=role,
              predictor_cls=DeepARPredictor)

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

### Make predictions and plot results

In [ ]:
freq = 'H'
prediction_length = 48
context_length = 72
t0 = '2016-01-01 00:00:00'
data_length = 400
num_ts = 200
period = 24

time_series = []
for k in range(num_ts):
    level = 10 * np.random.rand()
    seas_amplitude = (0.1 + 0.3*np.random.rand()) * level
    sig = 0.05 * level # noise parameter (constant in time)
    time_ticks = np.array(range(data_length))
    source = level + seas_amplitude*np.sin(time_ticks*(2*np.pi)/period)
    noise = sig*np.random.randn(data_length)
    data = source + noise
    index = pd.date_range(start=t0, freq=freq, periods=data_length)
    time_series.append(pd.Series(data=data, index=index))

time_series_training = []
for ts in time_series:
    time_series_training.append(ts[:-prediction_length]) 

Now we can use the previously created `predictor` object. For simplicity, we will predict only the first few time series used for training, and compare the results with the actual data we kept in the test set.

In [ ]:
predictor.set_prediction_parameters(freq, prediction_length)

Now we can use the `predictor` object to generate predictions.

In [ ]:
list_of_df = predictor.predict(time_series_training[:5])
actual_data = time_series[:5]

In [ ]:
for k in range(len(list_of_df)):
    plt.figure(figsize=(12,6))
    actual_data[k][-prediction_length-context_length:].plot(label='target')
    p10 = list_of_df[k]['0.1']
    p90 = list_of_df[k]['0.9']
    plt.fill_between(p10.index, p10, p90, color='y', alpha=0.5, label='80% confidence interval')
    list_of_df[k]['0.5'].plot(label='prediction median')
    plt.legend()
    plt.show()